![logo](https://resolvephilly.org/themes/custom/resolvephl-ci/logo.svg)

# Scraping and standardizing Pennsylvania Act 146 Quarterly Reports

**Author:** Julie Christie | Director of Data & Impact

**Partnering Team:** Our Kids

**Date:** March 28, 2024

## Background

### Goal of Analysis

Pennsylvania's Act 146 reports have a fairly standardized structure to how they are written. Therefore, this code will attempt to scrape the narrative `.pdf` documents and turn them into structured data.

Specifically, Resolve is looking to understand the frequency at which children who experience abuse that results in their death/near death are already known to the system.

### Glossary

-   **Act 146** -- *"Act 146 of 2006 went into effect on May 8, 2007. A major provision of this law requires that the department prepare a non-identifying summary for the governor and the General Assembly of findings for each case of substantiated child abuse or neglect that has resulted in a child fatality or near fatality."*
-   **Near fatality** -- *Definition TKTK*

### Data

-   [Quarterly Summaries of Child Fatalities/Near Fatalities](https://www.dhs.pa.gov/docs/Publications/Pages/Quarterly-Summaries-Child-Abuse.aspx) --- *A narrative `.pdf` document that describes the fatalities and near fatalities of children as a result of abuse or neglect.* \| [Metadata](URL)

### Tools

-   [Python](python.org) -- *Base code to facilitate scraping*
-   [Pandas](https://pandas.pydata.org/) -- *More robust data anlysis*
-   [Regex](https://developers.google.com/edu/python/regular-expressions) -- *Regular Expressions, or Regex, to parse out patterns of characters*

### Limitations
- A "certifying pysician" makes an individual call on whether a child's death/near death is the result of abuse, meaning that human error may result in cases not being documented in these reports
- Child fatalities and near fatalities as a result of abuse are an incredibly small and extreme subset of the overall abuse that children face.

## Cleaning

1. Develop Regex patterns for paragraphs. Below is an approximation of the structure of the `.pdf` files.

```
header header header bullshit
more header bs

Fatalities

County Name

1. A [age] [sex] person [died/nearly died] on [date]. [Agency] documented the case on [date] and identified [perpetrator(s)]. Details details details details for many sentences. [Family known or not known to agency before incident]

2. A [age] [sex] person [died/nearly died] on [date]. [Agency] documented the case on [date] and identified [perpetrator(s)]. Details details details details for many sentences. [Family known or not known to agency before incident]

County Name

3. A [age] [sex] person [died/nearly died] on [date]. [Agency] documented the case on [date] and identified [perpetrator(s)]. Details details details details for many sentences. [Family known or not known to agency before incident]

Near Fatalities

County Name

4. A [age] [sex] person [died/nearly died] on [date]. [Agency] documented the case on [date] and identified [perpetrator(s)]. Details details details details for many sentences. [Family known or not known to agency before incident]

Page number, footer bullshit
```